In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from PIL import Image
import matplotlib.image as mpimg
%matplotlib inline



In [2]:
## Importing our Images
ellipse =  'BW_Images/BW_Ellipse/'
spiral = 'BW_Images/BW_Spiral/'
path_sp = os.listdir(spiral)
path_el = os.listdir(ellipse)
print len(path_sp)
print len(path_el)
##### Building a Training Data
el_train = []
for i in range(1,100):
    im = mpimg.imread(ellipse + path_el[i])
    el_train.append(np.ravel(im))
sp_train = [] 
for i in range(1,100):
    im = mpimg.imread(spiral + path_sp[i])
    sp_train.append(np.ravel(im))
## Creating a training set
y_el_train = []
y_sp_train = []
for i in range(0,len(el_train)):
    y_el_train.append([0,1])
for i in range(0,len(sp_train)):
    y_sp_train.append([1,0])
X_train = el_train + sp_train
y_train = y_el_train + y_sp_train

######## Building the Test Data
el_test = []
sp_test = []
for i in range(1780,1800):
    im = mpimg.imread(ellipse + path_el[i])
    el_test.append(np.ravel(im))

for i in range(1780,1800):
    im = mpimg.imread(spiral + path_sp[i])
    sp_test.append(np.ravel(im))
## Creating a training set
y_el_test = []
y_sp_test = []
for i in range(0,len(el_test)):
    y_el_test.append([0,1])
for i in range(0,len(sp_test)):
    y_sp_test.append([1,0])
X_test = el_test + sp_test
y_test = y_el_test + y_sp_test






190215
62180


In [3]:
# Preparing our CNN
sess = tf.InteractiveSession()
x = tf.placeholder("float", shape = [None, 10000])
y_ = tf.placeholder("float", shape = [None, 2])
### Lets build a multi layer network
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)
### Convolution and Pooling
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides = [1,2,2,1], padding = 'SAME')


In [4]:
### First Convolutional Layer
W_conv1 = weight_variable([100,100,1,32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,100,100,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## The Information of our Parameters
print W_conv1
print b_conv1
print x_image
print h_conv1
print h_pool1

Tensor("Variable/read:0", shape=(100, 100, 1, 32), dtype=float32)
Tensor("Variable_1/read:0", shape=(32,), dtype=float32)
Tensor("Reshape:0", shape=(?, 100, 100, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 100, 100, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 50, 50, 32), dtype=float32)


In [5]:
## Second Convolutional layer
### Second Convolutional Layer
W_conv2 = weight_variable([100,100,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Densely Connected Layer
W_fc1 = weight_variable([25 * 25 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 25 * 25 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


In [ ]:
### Dropout To prevent overfitting
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# The Dimensions of our parameter
print W_conv2
print b_conv2
print h_conv2
print h_pool2
print h_pool2_flat


Tensor("Variable_2/read:0", shape=(100, 100, 32, 64), dtype=float32)
Tensor("Variable_3/read:0", shape=(64,), dtype=float32)
Tensor("Relu_1:0", shape=(?, 50, 50, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 25, 25, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 40000), dtype=float32)


In [ ]:
# Computing and Training
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(200):
    batch_xs = X_train
    batch_ys = y_train
    if i%1 == 0:
        print i
        train_accuracy = accuracy.eval(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    
    train_step.run(feed_dict={x: X_test, y_: y_test, keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={x: X_test, y_: y_test, keep_prob: 1.0}))


0
